<a href="https://colab.research.google.com/github/jsanmigimeno/FYP-Pix2Pix/blob/master/notebooks/FYP_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive
By default, generated files are designed to be saved to a folder named *FYP* on a mounted Google Drive account. If it is not desired to use a Google Drive account, change the variable *rootDir* to point to a valid directory where to save the generated files.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Default dir
rootDir = "/content/gdrive/My\ Drive/FYP"
!mkdir rootDir

# Clone Pix2Pix and install requirements
Download the latest available adapted Pix2Pix implementation, and install any requirements.

In [0]:
# Clone code
branch = 'master'
print("Cloning branch %s" % branch)
!git clone --single-branch --branch $branch https://github.com/jsanmigimeno/FYP-Pix2Pix.git
  
# Install requirements
%cd FYP-Pix2Pix
!pip install -r requirements.txt

## Update Local Code
If desired, download a new version of the code without deleting any downloaded datasets

In [0]:
print("Updating code from branch %s" % branch)
% cd ..
# Move dataset folder outside code directory
!mv ./FYP-Pix2Pix/DatasetsData ./
# Remove code directory
!rm -rf FYP-Pix2Pix/
# Clone new code
!git clone --single-branch --branch $branch https://github.com/jsanmigimeno/FYP-Pix2Pix.git
# Move dataset folder back into code directory
!mv ./DatasetsData ./FYP-Pix2Pix
%cd FYP-Pix2Pix

# Get dataset and set network models paths
Download dataset and set urls to pre-trained model. If using a custom trained model, add url to the saved model as a new entry on the *modelPaths* variable, together with the epoch number of the saved model.

In [0]:
# Download dataset
dataset = 'DSLog'  # 'Downscaled_All_JPEG' or 'DSLog'

!mkdir ./DatasetsData/
!python ./scripts/DatasetURLs.py -p './DatasetsData/' -d $dataset

In [0]:
# Pre trained models
modelPaths = {
    
## Baseline models
    'Baseline'             : (181, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EYrjP7urXEJFqg70-aFY4dABo5xj140jvlfJ8iC1pD_jQg?download=1'),
    'BaselineOnlyGAN'      : (160, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EXcK5lJTWwNLqX-4nTbRnbUBGz-Y-Pexr-yEeAke7YxBwQ?download=1'),
    'BaselineOnlyL1'       : (175, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/ET0Bitm4Y1hDkzSI8TpX6OMBSB34bTgQZAP1MCFWIPEq1A?download=1'),
    'BaselineOnLog'        : (168, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EWz7vp9buCdAmjvEw8hLRFoBSaSos3Z2pn_Ua4pWQnQoow?download=1'),
    
## Descriptor models
    # SIFT descriptor
    'DescriptorSIFT100'    : (188, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EUBShr3HezBMm1pOQrSw330B6Cqml5Id8l85Wj2mzk4J5w?download=1'),
    
    # HardNet descriptor
    'Descriptor100'        : (117, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EcF4f-YVY5ZNm-ilndb0evQB2qTNUj5bPQ8QsfZrA2YqUg?download=1'),
    'Descriptor150'        : (186, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EXaY5sbKGLVOhsMLEQNBdnIBnZiuvbtkGdpVG_u0OJQzcw?download=1'),
    'Descriptor200'        : (195, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EaArj8lG-uRDgfBiwyPJQ5wB-Lz-E6Fxq2LZCibOmzW0Xg?download=1'),
    'Descriptor300'        : (181, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EVwoo5GsGpVCuK2PmKqUFikBQl0gEEirS62wReACixD9Zw?download=1'),
    
    # HardNet and GAN only (no L1)
    'DescriptorGAN100'     : (34,  'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/ES9Je9a-bsdOrlR-_dJeVPgBYfCDiZOPP0KTx6p-Q4jy3g?download=1'),
    
    # Experiments: RGB patches (C) and No empty patches (NEP)
    'Descriptor100C'       : (91,  'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EQ5rZuYcHPdLuqSuVEHgJVoBmrrMPLmGgjqgK_zyK2Qlzw?download=1'),
    'Descriptor100NEP'     : (168, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EQXpCJNcGL1GswwuAzQjNf8B9fSbSwVMNyNWRswR5FMg1g?download=1'),
    
    # Trained on log transformed dataset
    'DescriptorOnLog100'   : (86,  'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/Ea0UrCiTv3BPsX8Tmuy_5n0Bh6nP-hC7IGVGUnB9uOfbLA?download=1'),
    'DescriptorOnLog200'   : (184, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EX_uVL0laIpKiDVsMXrx-WIBGhxyb4NGsYS4-obJvwmz0Q?download=1'),
    'DescriptorOnLog300'   : (166, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/Ed5bQgQiWZNHpl0kHrbJWt4B72RooWG3GnhWkQd9rk-HVg?download=1'),
    'DescriptorOnLog400'   : (120, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EaR3ackaONdEkErjD2h2F-UB06ACBzUz1lTFnT-h_VAhuQ?download=1'),
    'DescriptorOnLog500'   : (114, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/Ecbh7cLnAkVDh9wHVTPH9eAB4uq9gSkC1lr-dRRWYlhwTw?download=1'),
    
## Siamese models
    # SIFT descriptor
    'SiameseSIFT100'       : (145, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/Ea1QkcT4SoBHv2V99laCU6UBTjaPaEih81m1eSdWZXAY_w?download=1'),
    
    # HardNet descriptor
    'Siamese100'           : (172, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EbJ6gPF-FxBAs-X68cpLiZYBjd287IE6M3Ahj7L8eS8K_g?download=1'),
    
    # HardNet and GAN only (no L1)
    'SiameseGAN100'        : (187, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EYszxKPXmJBJow4_Tf3npKwBOai_E8TK7-YcA-WDGV7R4Q?download=1'),
    
    # Experiments: RGB patches (C)
    'Siamese100C'          : (192, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EVlyvOFGK95Hk_8ANQE8LjEB4lC9z5eLoU0ZeX2NfstArQ?download=1'),
    
    # Trained on log transformed dataset
    'SiameseOnLog100'      : (193, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EdIIXgYJdptPmNBiJJjub_YBL1fI2FUs61ZyNzAdFZjEUQ?download=1'),
    'SiameseOnLog200'      : (195, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EQbt6u9uTs5HoeVmwUhPp4wBxFzH0f1NjpdI6HQw5CaAiA?download=1'),
    'SiameseOnLog300'      : (195, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/ETH9Q5g6pqhDthSQ-ChknzUB0PMNN-OIh7KPkDPEgzpXLw?download=1'),
    'SiameseOnLog400'      : (197, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/ET19_3Pajl9AnjMqQLc-v1oBRffm5JZjd_cv3dpnQAEZPg?download=1'),
    'SiameseOnLog500'      : (161, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EVDyuEhobTJMh28wb4o670cBaSroaTpC19S2SrkoLEVj5w?download=1'),
    'SiameseOnLog100C'     : (113, 'https://imperiallondon-my.sharepoint.com/:u:/g/personal/js5415_ic_ac_uk/EZQzcTMesaFKiaOkAtmkouwB8OVPeQShb8tgb18q5l4_mA?download=1'),
}

# Test

In [0]:
from util.GDDownload import download_from_gd as dgd
from util.OneDownload import download_from_onedrive as dod
import os

def paramsListToString(paramsList):
  m = ""
  for param in paramsList:
    m += str(param) + " "
  return m

In [0]:
modelNames = ['Baseline']
  
bypassInference = False  # Bypass inference (to obtain metrics from unmodified dataset)
  
for name in modelNames:
  epoch = modelPaths[name][0]
  url = modelPaths[name][1]
  !mkdir $name
  modelFolder = './' + name
  modelPath = name + '/' + str(epoch) + '_net_G.pth'

  if os.path.isfile(modelPath):
    print("File already on disk, skipping download")
  else:
    print("Downloading file....")
    if 'google' in url:
      dgd(url, modelPath)
    else:
      dod(url, modelPath)
    print("Finished downloading")

  # Parameters
  testParams = [
      # Basic model parameters
      "--model", "pix2pix",                           # Model pix2pix
      "--checkpoints_dir", './',                      # Path to load checkpoints from
      "--results_dir",  rootDir+'/Results_'+dataset,  # Path to which save results
      "--dataroot", './DatasetsData/' + dataset,      # Dataset path
      "--name", name,                                 # Training name
      "--use_detector",
      "--num_points", 1024,

      # Dataset parameters
      "--dataset_mode", "aligned",                    # Aligned dataset mode (AB concatenated)
      "--load_size", 2048,                            # Resolution to which load images with (width) (default: 2120)
      "--crop_size", 2048,                            # default: 2120
      "--aspect_ratio", 1.4971751412,                 # Output images aspect ratio
      "--save_fake_only",
      #"--dataset_split", './split.csv',
      #"--log_trans",

      # Test parameters
      "--num_test", 598,                              # Number of test images to process (test size: 598)
      "--epoch", epoch,                               # Epoch to load
  ]
  
  if bypassInference:
    testParams = testParams + ["--bypass_inference"]

  testParamsString = paramsListToString(testParams)

  !python test.py $testParamsString

# Zip results

In [0]:
%cd /content/gdrive/My\ Drive/FYP/Results
for name in modelNames:
  saveName = name + '.zip'
  print("Zipping %s" % saveName)
  !zip -r $saveName $name
%cd /content/FYP-Pix2Pix/